#### All is well

Tempat mencoba2 hal *baru*
Compute2: input cycle,gparams dan rpm. mflow adalah salah satu output


In [27]:
# IMPORTS
import numpy as np
import CoolProp
from CoolProp.CoolProp import PropsSI as Props
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
#INPUT TURBINE INLET & OUTLET FLUID STATE
def whichcycle (k):
    global T_1, T_5, P_1, P_5, fluid #Temperature (K),Pressure(P),'Nama fluida'
    if k == 1:
        T_1 = 360   
        T_5 = 335.4     #This value is obtained from Refprop. Will be replace by value from CoolProp
        P_1 = 0.9   
        P_5 = 0.4   
        fluid = 'R245fa'
        mflow = 0.5
    if k == 2:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5
    if k == 3:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5
    if k == 4:
        T_1 = 0
        T_5 = 0
        P_1 = 0
        P_5 = 0
        fluid = 0 
        mflow = 0.5
    if k == 9:
        T_1 = 369.12
        P_1 = 3.0544    *1e6
        T_5 = 316.32
        P_5 = 0.94306   *1e6
        fluid = 'R1234yf' 
               

In [8]:

whichcycle(1)
sa1 = Props('S','T',T_1,'P',P_1*1e6,'R245FA')
T_5 = Props('T','P',P_5*1e6,'S',sa1,fluid) 
print(T_5)

335.5694173801896


In [6]:
img = Image.open('image.png')

477.34959677377134


In [40]:
T04     = 333.17
p04     = 1.6425 *1e6
T05ss     = 316.32
p05ss   = 0.94298 *1e6
# mflow   = 1.846
fluid   = 'R1234yf'
rho05ss = Props('D','T',T05ss,'P',p05ss,fluid)
rho04 = Props('D','T',T04,'P',p04,fluid)
h04ss   = Props('H','T',T05ss,'P',p05ss,fluid)
s04ss   = Props('S','T',T05ss,'P',p05ss,fluid)
fcoeff  = 0.25
wcoeff  = 0.92
r4      = 0.04
b4      = 0.008
U4      = r4*np.radians(20000)
Cm5     = fcoeff*U4
h5      = h05ss-1/2*Cm5**2


T04     = 333.17
p04     = 1.6425 *1e6
T05ss     = 316.32
p05ss   = 0.94298 *1e6
mflow   = 1.846
fluid   = 'R1234yf'
rho05ss = Props('D','T',T05ss,'P',p05ss,fluid)
rho04 = Props('D','T',T04,'P',p04,fluid)
h04ss   = Props('H','T',T05ss,'P',p05ss,fluid)
s04ss   = Props('S','T',T05ss,'P',p05ss,fluid)
r4      = 0.04
b4      = 0.008
C4_0    = 0
rho4ss_0= rho04


In [79]:

def whichgparam (l):
    global Rr5r4,Rb5b4,Rb4r4,RZrr4,rpm,NR
    if l == 1:
        Rr5r4   = 0.25
        Rb5b4   = 2.5
        Rb4r4   = 1/6
        RZrr4   = 1
        rpm     = 20000
        NR      = 10
    if l == 2:
        Rr5r4   = 0.25
        Rb5b4   = 2.5
        Rb4r4   = 1/6
        RZrr4   = 1
        rpm     = 30000
        NR      = 10

In [88]:

def Compute2(tenflow_coeff,tenwork_coeff,k,l): #INPUT 10 X flow atau work coeff
    #Input: state inlet-outlet,flow(2)&work coeff,gparams(r5/r4)
    global  T_1,T_5,P_1,P_5,p04s,p05ss,T04s,T05ss,h04,h05,h05ss,h5,h5ss,h01,DeltaH
    global  T4,T4s,p4,p4s,T5,p5,h4s,h4,h04s
    global  NR,r4,r5,rh5,rs5,b4,b5,Zr,Rb5b4
    global  C4,Ct4,Cm4,W4,U4,Alpha4,Beta4,C5,Ct5,Cm5,W5,U5,Alpha5,Beta5,Beta4opt,Beta4opt2
    global  Cm4didconverge1,Cm4didconverge2,k1Cm4,k2Cm4,errorC5
    global  rho4,rho4s,rho04,rho04s,a4s,a5ss,Ma4,Ma4ss
    global TotalLoss,LossInc,LossPass,LossTip,LossWind,LossTE,LossExit,rho4m,S5,O5,LossInc0
    global Effts,Efftt,Efftspred,Reaction,Effts,choked4,errorCm4
    global s04s,T04,T04s,p04,p04s,s04,etaspred,rho4sm,rho4,rho04,a5ss,rho5ss,p5ss,k1Cm5,k2Cm5
    global work_coeff,flow_coeff, Effreductbladeloading,tb4,tb5,mflow

    flow_coeff=tenflow_coeff/10
    work_coeff=tenwork_coeff/10
    whichcycle (k)       # The cycle to be computed is the k
    whichgparam (l)
    Cp4 = Props('C','T',T_1,'P',P_1,fluid)
    Cv4 = Props('O','T',T_1,'P',P_1,fluid)
    gamma = Cp4/Cv4
    Rx = 8.31446261815324   #J/K.mol

    #General Properties inlet outlet turbin (Total)
    H_1     = Props('H','T',T_1,'P',P_1,fluid)     #J/kg
    s01     = Props('S','T',T_1,'P',P_1,fluid)     #J/kg.K 
    T_5     = Props('T','P',P_5,'S',s01,fluid)  # =>asumsi nozzle isenthalpy DAN Isentropic
    H_5     = Props('H','T',T_5,'P',P_5,fluid)  # meski pada kenyataannya isenthalpic nozzle tidak isentropic
    DeltaH  = H_1-H_5            #Ideal === Isentropic Total Enthalpy change 

    C0s     = np.sqrt(2*DeltaH)         #Spouting Velocity

    #Perhitungan Properties ideal lain (Total)
    p01     = P_1           #inlet volute [1], Total
    T01     = T_1
    h01     = H_1
    p1      = p01           # inlet turbine, V~0 
    T1      = T_1
    h01     = H_1
    rho1   = Props('D','P',p1,'T',T1,fluid)
    h02s    = H_1           #inlet nozzle [2], Total
    s02s    = s01            #ideal volute === approx. as isentropic
    p02s    = p01
    T02s    = T01
    h03s    = h02s           #outlet nozzle [3], Total
    s03s    = s02s            #ideal nozzle === approx. as isentropic (in Total)
    p03s    = p02s
    T03s    = T02s
    h04s    = h03s           #inlet rotor [4], Total
    s04s    = s03s           #outlet nozzle === inlet rotor
    p04s    = p03s
    T04s    = T03s
    h04     = h04s          # Nozzle isenthalpic but not isentropic
    p05ss   = P_5
    T05ss   = T_5
    h05ss   = H_5
    s05ss   = s04s

    #Segitiga Kecepatan , m/s, radians
    U4      = np.sqrt(DeltaH/work_coeff)
    Ct4     = DeltaH/U4
    Cm5     = flow_coeff*U4
    h5ss    = h05ss-1/2*Cm5**2 # => Ct5=0, jadi C=Cm
    rho5ss  = Props('D','H',h5ss,'S',s01,fluid)
    rho05ss = Props('D','H',h05ss,'S',s01,fluid)
    # mflow   = rho5ss*2*np.pi*b5*r5*Cm5
    rho04s  = Props('D','H',h04,'S',s01,fluid)
    Cm4_0    = 0
    rho4s_0= rho04s        # => initial value for iteration
    Cm4ii    = Cm4_0
    rho4sii= rho4s_0
    Cm4didconverge1 = False
    Cm4didconverge2 = False
    choked4     = False
    k1Cm4    = 0
    k2Cm4    = 0
    while Cm4didconverge1 == False:
        k1Cm4       = k1Cm4+1             # => iteration amount
        Cm4i        = Cm4ii
        rho4si      = rho4sii
        Cm4ii       = Rb5b4*Rr5r4*(rho5ss/rho4si)*Cm5
        h4s         = h04s-1/2*(Cm4ii**2+Ct4**2)
        rho4sii     = Props('D','H',h4s,'S',s04s,fluid)
        errorCm4    = np.abs(Rb5b4*Rr5r4*(rho5ss/rho4sii)*(Cm5/Cm4ii)-1)
        # errorCm4    = mflow/(rho5ssii*Cm4ii*2*np.pi*b4*r4)-1
        if errorCm4 <= 10**-10:
            Cm4didconverge1 = True
            Cm4didconverge2 = True
            Cm4     = Cm4ii
            rho4s   = rho4sii
            break
        if (rho4si*Cm4i-rho4sii*Cm4ii)*(Cm4i-Cm4ii)<0:
            Cm4      = Cm4ii
            rho4s  = rho4sii
            break
    while Cm4didconverge2 == False:
        k2Cm4     = k2Cm4 +1         # => iteration amount
        Cm4       = Rb5b4*Rr5r4*(rho5ss/rho4s)*Cm5
        h4s       = h04ss-1/2*(Cm4ii**2+Ct4**2)
        rho4s     = Props('D','H',h4s,'S',s04s,fluid)
        if np.abs(1-Cm4/Props('A','H',h4s,'S',s04s,fluid)) < 5*1e-3:
            choked4 = True
            break
        rho4s     = Props('D','H',h4s,'S',s04s,fluid)
        errorCm4    = np.abs(Rb5b4*Rr5r4*(rho5ss/rho4s)*(Cm5/Cm4)-1)
        if errorCm4 <= 10**-10:
            Cm4didconverge2 = True
            break
    h4s     = h04s-1/2*(Cm4**2+Ct4**2)
    C4      = np.sqrt(Cm4**2+Ct4**2)
    W4      = np.sqrt(Cm4**2+(U4-Ct4)**2)
    Alpha4  = np.arccos(Cm4/C4)
    Beta4   = np.arccos(Cm4/W4)
    Alpha5  = 0
    Ct5     = Cm5*np.tan(Alpha5)
    U5      = U4*Rr5r4
    W5      = np.sqrt(Cm5**2+(U5-Ct5)**2)
    C5      = np.sqrt(Ct5**2+Cm5**2)
    Beta5   = np.arccos(Cm5/W5)

    #Perhitungan geometri
    r4      = U4/np.radians(rpm*6)
    r5      = Rr5r4*r4
    b4      = Rb4r4*r4
    b5      = Rb5b4*b4
    rs5     = (2*r5+b5)/2
    rh5     = rs5-b5
    if rh5 < 0.0015:
        print("For flow coeff =",flow_coeff,"and work coeff=",work_coeff,"rh5 too small(<1.5mm, adjust gparams")
        return
    Zr      = RZrr4*r4

    mflow   = 2*np.pi*b5*r5*rho5ss*Cm5

    Q5      = mflow/rho05ss
    ns      = np.radians(rpm*6)*np.sqrt(Q5)/DeltaH**0.75
    Efftspred    = 0.81-1.07*(ns-0.55)**2-0.5*(ns-0.55)**3       #predicted total-to-static efficiency

    

    p04     = p01-rho1*DeltaH*(1-Efftspred)/4   #predicting loss due to nozzle
    T04     = Props('T','P',p04,'H',h04s,fluid)
    s04     = Props('S','P',p04,'T',T04,fluid)
    

    #Perhitungan Properties ideal lain (Static)
    h4s     = h04s-1/2*C4**2
    p4s     = Props('P','H',h4s,'S',s04s,fluid)
    p4      = Props('P','H',h4s,'S',s04,fluid)
    T4s     = Props('T','H',h4s,'S',s04s,fluid)
    rho04s  = Props('D','P',p04s,'T',T04s,fluid)
    rho4s   = Props('D','P',p4s,'T',T4s,fluid)
    rho4sm  = 2*(p04s-p4s)/C4**2
    h4      = h04-1/2*C4**2
    p4      = Props('P','H',h4,'S',s04,fluid)
    T4      = Props('T','H',h4,'S',s04,fluid)
    rho04   = Props('D','P',p04,'T',T04,fluid)
    rho4    = Props('D','P',p4,'H',h04,fluid)
    rho4m   = 2*(p04-p4)/C4**2
    a01     = Props('A','P',p01,'T',T01,fluid)
    a4s     = Props('A','P',p4s,'T',T4s,fluid)
    a4      = Props('A','P',p4,'T',T4,fluid)
    Ma4s    = C4/a4s
    Ma4     = C4/a4


    Re4s    = rho4s*Cm4*b4/Props('V','P',p4s,'T',T4s,fluid)
    # Re4     = rho4*C4*b4/Props('V','P',p4,'T',T4,fluid)



    S5      = 2*np.pi*r5/NR
    O5      = S5*Cm5/W5

    p5ss    = Props('P','H',h5ss,'S',s05ss,fluid)
    a5ss    = Props('A','H',h5ss,'P',p5ss,fluid)
    Ma5ss   = C5/a5ss


    # \\\\\\\ <<---------<<----||----->>------------>> ////////
    ## Losses Coefficient ##

    #Rotor Incidence Losses (Chen)
    Beta4opt2 = np.arctan((-1.98/NR)/(1-1.98/NR)*np.tan(Alpha4)) #
    Beta4opt  = np.arctan(np.tan(Alpha4)*(work_coeff-1+2/NR)/work_coeff)    #(Chen)
    LossInc0  = 0.5*(W4**2)*(np.sin(np.abs(np.abs(Beta4)-np.abs(Beta4opt))))**2  #m2/s2
    LossInc   = 0.5*(W4**2)*(np.sin(Beta4)-np.sin(Beta4opt))**2
    
    #Blade loading efficiency (Chen) 
    vNu = U4/np.sqrt(2*Cp4*T01*(1-(p5ss/p01)**((gamma-1)/gamma))) #blade/isentropic jet speed ratio
    Effreductbladeloading = flow_coeff**2*vNu**2

    #Rotor Passage Losses([Uusitalo] from Moustapha. PLM3)
    LH = np.pi/4*((Zr-b4/2)+(r4-rh5-b5/2))                                                              #m
    DH = 0.5*((4*np.pi*r4*b4/(2*np.pi*r4+Zr*rh5))+((2*np.pi*(rs5**2-rh5**2)/(np.pi*(rs5-rh5))+Zr*b5)))  #m
    Y5 = np.arctan(0.5*(np.tan(Beta4)+np.tan(Beta5)))
    C = Zr/np.cos(Y5)
    if (r4-rs5)/b5>=0.2:
        KpCETI = 0.11
    else:
        KpCETI = 0.22
    LossPass = KpCETI*(LH/DH+0.68*((1-(r5/r4)**2)*np.cos(Beta5)/(b5/C))*((W4**2+W5**2)/2))
    
    #Rotor Clearance Losses
    Ca = (1-(rs5/r4))/(Cm4*b4)
    Cr = (rs5/r4)*((Zr-b4)/(Cm5*r5*b5))
    Ka = 0.4
    Kr = 0.75
    Kar = -0.3
    Ea = 0.0003
    Er = 0.0003
    if Ea*Er*Ca*Cr>=0:
        LossTip = (U4**3*NR/(8*np.pi))*(Ka*Ea*Ca+Kr*Er*Cr+Kar*np.sqrt(Ea*Er*Ca*Cr))
    else:
        LossTip = (U4**3*NR/(8*np.pi))*(Ka*Ea*Ca+Kr*Er*Cr)
        
    #Windage Losses
    Eb = 0.0003
    Kf = 3.7*(Eb/r4)**0.1/Re4s**0.5
    LossWind = Kf*((rho4s+rho5ss)/2)*U4**3*r4**2/(2*mflow*W5**2)

    #Trailing Edge Losses
    tb4 = 0.04*r4
    tb5 = 0.02*r4
    LossTE = rho5ss*W5**2/2*(NR*tb5/(np.pi*(rh5+rs5)*np.cos(Beta5)))**2
    
    #Exit Losses
    LossExit = 0.5*C5**2    # => mungkin untuk ubah dari total jadi static. abaikan dulu

    # => Sum Enthalpy Losses
    TotalLoss = LossInc + LossPass + LossTip + LossWind + LossTE
    # \\\\\\\ <<---------<<----||----->>------------>> ////////


    #Perhitungan Properties considering losses
    h05 = h05ss+ (LossInc0+LossPass+LossTip+LossWind+LossTE)           #bentar masih salah lossnya       #nozzle masih diasumsikan isentropic dan isenthalpic
    h5  = h5ss+ (LossInc+LossPass+LossTip+LossWind+LossTE) 
    p05 = p05ss
    T05 = Props('T','H',h05,'P',p05,fluid)
    p5  = p5ss
    T5  = Props('T','H',h5,'P',p5,fluid)

    #Effisiensi 
    Reaction    = (h4s-h5)/(h01-h05ss)
    Efftt       = ((h01-h05)/(h01-h05ss)-Effreductbladeloading)*100
    Effts       = ((h01-h05)/(h01-h5ss)-Effreductbladeloading)*100



In [86]:

Compute2(2,10,9,2)
print((LossInc,LossInc0,LossPass,LossTip,LossWind,LossTE))
print(Efftt,Effts)
print(Cm4didconverge1,Cm4didconverge2)
print(np.degrees(Beta4),np.degrees(Beta4opt),np.degrees(Beta4opt2))
print("r4=",r4*1000,"b4=",b4*1000)
print("rh5=",rh5*1000,"rs5=",rs5*1000,"b5=",b5*1000,"Zr=",Zr*1000)
print(errorCm4,k1Cm4,k2Cm4,Cm4,C4)

(39.05377079464954, 39.05377079464954, 132.64914867933024, 1298.2750581604425, 0.04663474456820707, 2117.4481279400115)
81.94648885917285 80.3374476517631
True True
0.0 71.79134306177949 -75.07839202051808
r4= 45.01389015523186 b4= 7.502315025871977
rh5= 1.875578756467995 rs5= 20.63136632114794 b5= 18.755787564679945 Zr= 45.01389015523186
2.772448937093941e-12 6 0 9.30373294144985 141.72102311727284


In [89]:
print(rpm)
print(DeltaH)
print(tb4,tb5)
print(U4,mflow)
print(np.degrees(Alpha4),np.degrees(Beta4),np.degrees(Alpha5),np.degrees(Beta5))
print(Effts,Reaction*100)
print(Ma4)

30000
19998.28894676076
0.0018005556062092745 0.0009002778031046373
141.41530662117435 1.8653919750042878
86.23592707384944 0.0 0.0 51.34019174590991
80.3374476517631 33.84468444621623
1.1482548124827154


In [30]:
import pandas as pd
dftes=pd.read_csv('x:/TA-Turbine-Design-13118082/Inputs/cyclelist.csv',skiprows=1,header=0,index_col=0)
dftes.head()
k=1
tess = dftes.iloc[0]['T_1']
T_1 = dftes.iloc[k-1]['T_1']
P_1 = dftes.iloc[k-1]['P_1']
T_5 = dftes.iloc[k-1]['T_5']
T_5 = dftes.iloc[k-1]['P_5']
fluid = dftes.iloc[k-1]['fluid']
print(Props('H','T',T_1,'P',P_1,fluid))

484816.69184496225


In [41]:
import pandas as pd
dfrpms=pd.read_csv('x:/TA-Turbine-Design-13118082/Inputs/rpmlist.csv',skiprows=0,header=0,index_col=0)
dfrpms.head()
rpmtes=dfrpms.iloc[2-1]['rpm']
print(f"rpm ke 2 adalah {rpmtes}")

rpm ke 2 adalah 25000


In [63]:
import pandas as pd
dffitfun = pd.read_csv('x:/TA-Turbine-Design-13118082/Inputs/fittedpolycoeffs.csv',skiprows=4,header=1,index_col=False)
dffitfun.head()
p=np.zeros(shape=(6, 6))
for i in range(0,6):
    for j in range(0,6):
        p[i][j]=dffitfun.iloc[i][j+1]
print(p[0][5])

9.0


In [49]:
p=np.zeros(shape=(6, 6))
p[0][0] =      -112.4 
p[1][0] =       29.54 
p[0][1] =       46.75 
p[2][0] =      -30.73 
p[1][1] =        7.22 
p[0][2] =      -5.305 
p[3][0] =       5.809 
p[2][1] =      0.2481 
p[1][2] =     -0.5259 
p[0][3] =      0.2841 
p[4][0] =     -0.6329 
p[3][1] =     0.04815 
p[2][2] =    -0.02632 
p[1][3] =     0.01951 
p[0][4] =   -0.007687 
p[5][0] =     0.02749 
p[4][1] =   -0.003921 
p[3][2] =    0.001028 
p[2][3] =    7.81e-05 
p[1][4] =  -0.0002179 
p[0][5] =   8.017e-05
print(p)

[[-1.124e+02  4.675e+01 -5.305e+00  2.841e-01 -7.687e-03  8.017e-05]
 [ 2.954e+01  7.220e+00 -5.259e-01  1.951e-02 -2.179e-04  0.000e+00]
 [-3.073e+01  2.481e-01 -2.632e-02  7.810e-05  0.000e+00  0.000e+00]
 [ 5.809e+00  4.815e-02  1.028e-03  0.000e+00  0.000e+00  0.000e+00]
 [-6.329e-01 -3.921e-03  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 2.749e-02  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]]


In [ ]:
import os
Rutedir = os.path.dirname(os.path.abspath('__file__'))
print(Rutedir)
import pandas as pd
dftessy=pd.read_csv(os.path.join(Rutedir,"/My Codes"))
dftessy.head()

In [13]:
import pandas as pd
import numpy as np
import os
from definitions import *

gparamlist=[[Rr5r4,'Rr5r4'],[Rb5b4,'Rb5b4'],[Rb4r4,'Rb4r4'],[RZrr4,'RZrr4'],[NR,'NR']]
dfgparams=pd.read_csv(os.path.join(ROOT_DIR,"Inputs\gparamslist.csv"),skiprows=1,header=0,index_col=0)
# for x in gparamlist:
#     x = dfgparams.iloc[l-1][y]

ModuleNotFoundError: No module named 'definitions'